REMEMBER: FIRST CREATE A COPY OF THIS FILE WITH A UNIQUE NAME AND DO YOUR WORK THERE. AND MAKE SURE YOU COMMIT YOUR CHANGES TO THE `hw3_submissions` BRANCH.

# Assignment 3 | Cleaning and Exploring Data with Pandas



<img src="data/scoreCard.jpg" width=250>

In this assignment, you will investigate restaurant food safety scores for restaurants in San Francisco. Above is a sample score card for a restaurant. The scores and violation information have been made available by the San Francisco Department of Public Health. 

## Loading Food Safety Data


There are 2 files in the data directory:
1. business.csv containing food establishments in San Francisco
1. inspections.csv containing retaurant inspections records

Let's start by loading them into Pandas dataframes.  One of the files, business.csv, has encoding (ISO-8859-1), so you will need to account for that when reading it.

### Question 1

#### Question 1a
Read the two files noted above into two pandas dataframes named `bus` and `ins`, respectively. Print the first 5 rows of each to inspect them.


In [1]:
import pandas as pd

In [3]:
bus = pd.read_csv('data/data/businesses.csv', encoding='ISO-8859-1')
ins = pd.read_csv('data/data/inspections.csv')

In [126]:
bus.head(5)

,business_id,name,address,city,state,postal_code,latitude,longitude,phone_number
0,10,TIRAMISU KITCHEN,033 BELDEN PL,San Francisco,CA,94104,37.791116,-122.403816,+14154217044
1,19,NRGIZE LIFESTYLE CAFE,"1200 VAN NESS AVE, 3RD FLOOR",San Francisco,CA,94109,37.786848,-122.421547,+14157763262
2,24,OMNI S.F. HOTEL - 2ND FLOOR PANTRY,"500 CALIFORNIA ST, 2ND FLOOR",San Francisco,CA,94104,37.792888,-122.403135,+14156779494
3,31,NORMAN'S ICE CREAM AND FREEZES,2801 LEAVENWORTH ST,San Francisco,CA,94133,37.807155,-122.419004,NaN
4,45,CHARLIE'S DELI CAFE,3202 FOLSOM ST,San Francisco,CA,94110,37.747114,-122.413641,+14156415051


In [127]:
ins.head(5)

,business_id,score,date,type,year
0,10,82,20160503,routine,2016
1,10,94,20140729,routine,2014
2,10,92,20140114,routine,2014
3,19,94,20160513,routine,2016
4,19,94,20141110,routine,2014


## Examining the Business data

From its name alone, we expect the `businesses.csv` file to contain information about the restaurants. Let's investigate this dataset.

### Question 2

#### Question 2a: How many records are there?

In [128]:
# there are 6,315 total rows/records
print(bus.shape)
print(bus.info())

(6315, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6315 entries, 0 to 6314
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   business_id   6315 non-null   int64  
 1   name          6315 non-null   object 
 2   address       6315 non-null   object 
 3   city          6315 non-null   object 
 4   state         6315 non-null   object 
 5   postal_code   6056 non-null   object 
 6   latitude      3599 non-null   float64
 7   longitude     3599 non-null   float64
 8   phone_number  5696 non-null   object 
dtypes: float64(2), int64(1), object(6)
memory usage: 444.1+ KB
None


#### Question 2b: How many unique business IDs are there?  

In [129]:
# there are 6,315 unique business IDs
bus['business_id'].describe()

count     6315.000000
mean     48562.416627
std      33825.711496
min         10.000000
25%       5891.500000
50%      65012.000000
75%      79197.000000
max      90239.000000
Name: business_id, dtype: float64

#### Question 2c: What are the 5 most common businesses by name, and how many are there in San Francisco?

In [130]:
# list of 5 most common businesses by name
bus.groupby(['name']).name.value_counts().nlargest(5)

name                        name                      
STARBUCKS COFFEE            STARBUCKS COFFEE              72
PEET'S COFFEE & TEA         PEET'S COFFEE & TEA           24
MCDONALDS                   MCDONALDS                     12
SAN FRANCISCO SOUP COMPANY  SAN FRANCISCO SOUP COMPANY    11
WALGREENS                   WALGREENS                     11
Name: name, dtype: int64

## Zip code

Next, let's  explore some of the variables in the business table. We begin by examining the postal code.

### Question 3

#### Question 3a
How are the zip code values stored in python (i.e. data type)?

To answer this you might want to examine a particular entry.

In [132]:
row2_zip = bus.iloc[1]['postal_code']

In [140]:
# based on the error message we can see that the particular entry is a 'str' object
row2_zip.dtype

AttributeError: 'str' object has no attribute 'dtype'

In [138]:
# the data type of the column is listed as 'object'
bus['postal_code'].dtype

dtype('O')

#### Question 3b

What are the unique values of postal_code?

In [141]:
# unique values are listed below, because it's an object even same zip codes are considered separate and unique values
bus['postal_code'].value_counts

<bound method IndexOpsMixin.value_counts of 0       94104
1       94109
2       94104
3       94133
4       94110
        ...  
6310    94103
6311    94127
6312    94116
6313    94112
6314    94110
Name: postal_code, Length: 6315, dtype: object>

#### Question 3c

Let's say we decide to exclude the businesses that have no zipcode for our analysis (which might include food trucks for example).  Use the list of valid 5-digit zip codes below to create a new dataframe called bus_valid, with only businesses whose postal_codes show up in this list of valid zipcodes. How many businesses are there in this new dataframe?

In [142]:
validZip = ["94102", "94103", "94104", "94105", "94107", "94108",
            "94109", "94110", "94111", "94112", "94114", "94115",
            "94116", "94117", "94118", "94121", "94122", "94123", 
            "94124", "94127", "94131", "94132", "94133", "94134"]

In [143]:
# there are 5,999 businesses in the new dataframe with valid zipcodes
validBus = bus[bus['postal_code'].isin(validZip)]
validBus.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5999 entries, 0 to 6314
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   business_id   5999 non-null   int64  
 1   name          5999 non-null   object 
 2   address       5999 non-null   object 
 3   city          5999 non-null   object 
 4   state         5999 non-null   object 
 5   postal_code   5999 non-null   object 
 6   latitude      3516 non-null   float64
 7   longitude     3516 non-null   float64
 8   phone_number  5444 non-null   object 
dtypes: float64(2), int64(1), object(6)
memory usage: 468.7+ KB


## Latitude and Longitude

Another aspect of the data we want to consider is the prevalence of missing values. If many records have missing values then we might be concerned about whether the nonmissing values are representative of the population.

### Question 4
 
Consider the longitude and latitude in the business DataFrame. 

#### Question 4a

How many businesses are missing longitude values, working with only the businesses that are in the list of valid zipcodes?

In [144]:
# 2,483 businesses are missing longitude values
validBus[validBus['longitude'].isnull()]

,business_id,name,address,city,state,postal_code,latitude,longitude,phone_number
200,821,LA SANTANECA,3781 MISSION ST,San Francisco,CA,94110,NaN,NaN,+14156481034
210,852,NEW HOLLYWOOD BAKERY & RESTAURANT,652 PACIFIC AVE,San Francisco,CA,94133,NaN,NaN,+14153979919
760,2934,LOU'S FISH SHACK,300 JEFFERSON ST,San Francisco,CA,94133,NaN,NaN,+14157715687
826,3123,LES JOULINS,44 ELLIS ST,San Francisco,CA,94102,NaN,NaN,+14153975397
1100,4323,DIVA'S,1081 POST ST,San Francisco,CA,94109,NaN,NaN,+14154743482
...,...,...,...,...,...,...,...,...,...
6310,90234,ACCESS SFUSD THE ARC,1520 HOWARD ST,San Francisco,CA,94103,NaN,NaN,NaN
6311,90236,PHO DONG HUONG,667 MONTEREY BLVD,San Francisco,CA,94127,NaN,NaN,+14154945048
6312,90237,WIN'S RESTAURANT,3040 TARAVAL ST,San Francisco,CA,94116,NaN,NaN,+14158606036
6313,90238,USA POWER MARKET,1524 OCEAN AVE,San Francisco,CA,94112,NaN,NaN,+14153331878


#### Question 4b

Create a new dataframe with one row for each valid zipcode. The dataframe should include the following three columns:

1. `postal_code`: Contains the zip codes in the `validZip` variable above.
2. `null_lon`: The number of businesses in that zipcode with missing `longitude` values.
3. `not_null_lon`: The number of businesses without missing `longitude` values.

In [145]:
# create data subset of just businesses without longitude values
null_lon1 = validBus[validBus['longitude'].isna()]
null_lon1

,business_id,name,address,city,state,postal_code,latitude,longitude,phone_number
200,821,LA SANTANECA,3781 MISSION ST,San Francisco,CA,94110,NaN,NaN,+14156481034
210,852,NEW HOLLYWOOD BAKERY & RESTAURANT,652 PACIFIC AVE,San Francisco,CA,94133,NaN,NaN,+14153979919
760,2934,LOU'S FISH SHACK,300 JEFFERSON ST,San Francisco,CA,94133,NaN,NaN,+14157715687
826,3123,LES JOULINS,44 ELLIS ST,San Francisco,CA,94102,NaN,NaN,+14153975397
1100,4323,DIVA'S,1081 POST ST,San Francisco,CA,94109,NaN,NaN,+14154743482
...,...,...,...,...,...,...,...,...,...
6310,90234,ACCESS SFUSD THE ARC,1520 HOWARD ST,San Francisco,CA,94103,NaN,NaN,NaN
6311,90236,PHO DONG HUONG,667 MONTEREY BLVD,San Francisco,CA,94127,NaN,NaN,+14154945048
6312,90237,WIN'S RESTAURANT,3040 TARAVAL ST,San Francisco,CA,94116,NaN,NaN,+14158606036
6313,90238,USA POWER MARKET,1524 OCEAN AVE,San Francisco,CA,94112,NaN,NaN,+14153331878


In [146]:
# create data subset of all other businesses with longitude values
not_null_lon1 = validBus[~validBus['longitude'].isna()]
not_null_lon1

,business_id,name,address,city,state,postal_code,latitude,longitude,phone_number
0,10,TIRAMISU KITCHEN,033 BELDEN PL,San Francisco,CA,94104,37.791116,-122.403816,+14154217044
1,19,NRGIZE LIFESTYLE CAFE,"1200 VAN NESS AVE, 3RD FLOOR",San Francisco,CA,94109,37.786848,-122.421547,+14157763262
2,24,OMNI S.F. HOTEL - 2ND FLOOR PANTRY,"500 CALIFORNIA ST, 2ND FLOOR",San Francisco,CA,94104,37.792888,-122.403135,+14156779494
3,31,NORMAN'S ICE CREAM AND FREEZES,2801 LEAVENWORTH ST,San Francisco,CA,94133,37.807155,-122.419004,NaN
4,45,CHARLIE'S DELI CAFE,3202 FOLSOM ST,San Francisco,CA,94110,37.747114,-122.413641,+14156415051
...,...,...,...,...,...,...,...,...,...
4025,71784,URBAN BREAD CAFE,3901 18TH ST,San Francisco,CA,94114,37.761113,-122.430789,+14155745682
4029,71823,SUBWAY SANDWICHES #24254,200 PINE ST #140,San Francisco,CA,94104,37.792237,-122.400124,+14152386388
4031,71841,"RAPHA RACHING, LLC",2198 FILBERT ST,San Francisco,CA,94123,37.798078,-122.435444,+14152063087
4033,71909,SHANGHAI DUMPLING KING 2,696 MONTEREY BLVD,San Francisco,CA,94127,37.731568,-122.450970,+14152382380


In [147]:
# group null longitude subset by zipcode
grouped_null_lon = null_lon1.groupby(null_lon1['postal_code']).count()
grouped_null_lon

,business_id,name,address,city,state,latitude,longitude,phone_number
postal_code,,,,,,,,
94102,196,196,196,196,196,0,0,166
94103,252,252,252,252,252,0,0,211
94104,59,59,59,59,59,0,0,54
94105,100,100,100,100,100,0,0,84
94107,247,247,247,247,247,0,0,220
94108,82,82,82,82,82,0,0,71
94109,146,146,146,146,146,0,0,121
94110,251,251,251,251,251,0,0,223
94111,115,115,115,115,115,0,0,102


In [148]:
# group not null longitude subset by zipcode
grouped_not_null_lon = not_null_lon1.groupby(not_null_lon1['postal_code']).count()
grouped_not_null_lon

,business_id,name,address,city,state,latitude,longitude,phone_number
postal_code,,,,,,,,
94102,262,262,262,262,262,262,262,239
94103,306,306,306,306,306,306,306,292
94104,74,74,74,74,74,74,74,65
94105,126,126,126,126,126,126,126,116
94107,204,204,204,204,204,204,204,192
94108,142,142,142,142,142,142,142,132
94109,237,237,237,237,237,237,237,216
94110,334,334,334,334,334,334,334,315
94111,167,167,167,167,167,167,167,150


In [149]:
# create new dataframe based on groupby series with count of businesses with null longitude values grouped by zipcode
missing_lon = pd.DataFrame(grouped_null_lon['business_id'])
missing_lon = missing_lon.rename(columns={'business_id': 'null_lon'})

In [151]:
# add new column to dataframe with count of business with longitude values also grouped by zipcode
missing_lon['not_null_lon'] = grouped_not_null_lon['business_id']
missing_lon

,null_lon,not_null_lon
postal_code,,
94102,196,262
94103,252,306
94104,59,74
94105,100,126
94107,247,204
94108,82,142
94109,146,237
94110,251,334
94111,115,167


#### 4c. Do any zip codes appear to have more than their 'fair share' of missing longitude? 

To answer this, you will want to compute the proportion of missing longitude values for each zip code, and print the proportion missing longitude, and print the top five zipcodes in descending order of proportion missing postal_code.


In [152]:
# print proportion missing longitude values
missing_lon['prop_missing_lon'] = missing_lon['null_lon'] / (missing_lon['null_lon'] + missing_lon['not_null_lon'])
missing_lon

,null_lon,not_null_lon,prop_missing_lon
postal_code,,,
94102,196,262,0.427948
94103,252,306,0.451613
94104,59,74,0.443609
94105,100,126,0.442478
94107,247,204,0.547672
94108,82,142,0.366071
94109,146,237,0.381201
94110,251,334,0.429060
94111,115,167,0.407801


In [153]:
# print top 5 with null longitude values
(missing_lon.sort_values(by='prop_missing_lon', ascending=False)).head(5)

,null_lon,not_null_lon,prop_missing_lon
postal_code,,,
94107,247,204,0.547672
94124,88,76,0.536585
94132,65,69,0.485075
94114,108,122,0.469565
94122,123,147,0.455556


# Investigate the inspection data

Let's now turn to the inspection DataFrame. Earlier, we found that `ins` has 4 columns, these are named `business_id`, `score`, `date` and `type`.  In this section, we determine the granularity of `ins` and investigate the kinds of information provided for the inspections. 

### Question 5

#### Question 5a
As with the business data, assess whether there is one inspection record for each business, by counting how many rows are in the data and how many unique businesses there are in the data. If they are exactly the same number, it means there is only one inspection per business, clearly.

In [154]:
# there are 15,430 rows in the inspection data
print(ins.shape)
print(ins.info())

(15430, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15430 entries, 0 to 15429
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   business_id  15430 non-null  int64 
 1   score        15430 non-null  int64 
 2   date         15430 non-null  object
 3   type         15430 non-null  object
 4   year         15430 non-null  object
dtypes: int64(2), object(3)
memory usage: 602.9+ KB
None


In [162]:
# there are 5,730 unique businesses in the inspection data which means there are more than one inspection for some businesses
ins['business_id'].nunique()

5730

#### Question 5b

What values does `type` take on? How many occurrences of each value is in the DataFrame? Create a new dataframe named `ins2` by copying `ins` and keeping only records with values of `type` that occur more than 10 times in the original table. In other words, eliminate records that have values of `type` that occur rarely (< 10 times). Check the result to make sure rare types are eliminated.

In [65]:
# there are 15429 occurences of routine in the DataFrame and 1 occurence of complaint in the DataFrame
ins['type'].value_counts()

routine      15429
complaint        1
Name: type, dtype: int64

In [168]:
# selected records with only the type occurence 'routine' which occurs more than 10 times 
# ommit 'complaint' which only occurs once
ins2 = ins[ins['type'].isin(['routine'])]
ins2

,business_id,score,date,type,year
0,10,82,20160503,routine,2016
1,10,94,20140729,routine,2014
2,10,92,20140114,routine,2014
3,19,94,20160513,routine,2016
4,19,94,20141110,routine,2014
...,...,...,...,...,...
15425,89515,98,20161103,routine,2016
15426,89701,100,20161206,routine,2016
15427,89790,96,20161129,routine,2016
15428,89803,100,20161215,routine,2016


In [169]:
# another option to select only type values that appear greater than 10 times in the dataset
vc = ins['type'].value_counts()
ins[ins['type'].isin(vc.index[vc.gt(10)])]

,business_id,score,date,type,year
0,10,82,20160503,routine,2016
1,10,94,20140729,routine,2014
2,10,92,20140114,routine,2014
3,19,94,20160513,routine,2016
4,19,94,20141110,routine,2014
...,...,...,...,...,...
15425,89515,98,20161103,routine,2016
15426,89701,100,20161206,routine,2016
15427,89790,96,20161129,routine,2016
15428,89803,100,20161215,routine,2016


#### Question 5c

Since the data was stored in a .csv file, the dates are formatted as strings such as `20160503`. Once we read in the data, we would like to have dates in an appropriate format for analysis. Add a new column called `year` by capturing the first four characters of the date column. 

Hint: we have seen multiple ways of doing this in class, includings `str` operations, `lambda` functions, `datetime` operations, and others. Choose the method that works best for you :)

In [163]:
# convert date column to string data type
ins['date'] = ins['date'].astype(str)

In [164]:
# create new column from first four chars of data column
ins['year'] = ins['date'].str[:4]
ins.head()

,business_id,score,date,type,year
0,10,82,20160503,routine,2016
1,10,94,20140729,routine,2014
2,10,92,20140114,routine,2014
3,19,94,20160513,routine,2016
4,19,94,20141110,routine,2014


#### Question 5d

What range of years is covered in this data set? Are there roughly same number of inspections each year? Try dropping records for any years with less than 50 inspections and store the result in a new dataframe named `ins3`.

In [165]:
# 2013 has less than 50 inspection records
ins.groupby(ins['year']).count()

,business_id,score,date,type
year,,,,
2013,38,38,38,38
2014,5629,5629,5629,5629
2015,3923,3923,3923,3923
2016,5840,5840,5840,5840


In [166]:
# data without 2013 stored in new dataframe
ins3 = ins[ins['year'].isin(['2014', '2015', '2016'])]
ins3

,business_id,score,date,type,year
0,10,82,20160503,routine,2016
1,10,94,20140729,routine,2014
2,10,92,20140114,routine,2014
3,19,94,20160513,routine,2016
4,19,94,20141110,routine,2014
...,...,...,...,...,...
15425,89515,98,20161103,routine,2016
15426,89701,100,20161206,routine,2016
15427,89790,96,20161129,routine,2016
15428,89803,100,20161215,routine,2016


Let's examine only the inspections for one year: 2016. This puts businesses on a more equal footing because [inspection guidelines](https://www.sfdph.org/dph/eh/Food/Inspections.asp) generally refer to how many inspections should occur in a given year.

In [167]:
# inspections for just 2016 stored in new dataframe
ins2016 = ins[ins['year'] == '2016']
ins2016

,business_id,score,date,type,year
0,10,82,20160503,routine,2016
3,19,94,20160513,routine,2016
6,24,98,20161005,routine,2016
7,24,96,20160311,routine,2016
10,31,84,20160816,routine,2016
...,...,...,...,...,...
15425,89515,98,20161103,routine,2016
15426,89701,100,20161206,routine,2016
15427,89790,96,20161129,routine,2016
15428,89803,100,20161215,routine,2016


### Question 6

#### Question 6a

Merge the business and 2016 inspections data, keeping all businesses regardless of whether they show up in the inspections file. Show the first several rows of the resulting dataframe.

In [107]:
# merge completed betwee business and 2016 inspections data
bus2016_merged = pd.merge(bus, ins2016, on="business_id", how="outer")
bus2016_merged

,business_id,name,address,city,state,postal_code,latitude,longitude,phone_number,score,date,type,year
0,10,TIRAMISU KITCHEN,033 BELDEN PL,San Francisco,CA,94104,37.791116,-122.403816,+14154217044,82.0,20160503,routine,2016
1,19,NRGIZE LIFESTYLE CAFE,"1200 VAN NESS AVE, 3RD FLOOR",San Francisco,CA,94109,37.786848,-122.421547,+14157763262,94.0,20160513,routine,2016
2,24,OMNI S.F. HOTEL - 2ND FLOOR PANTRY,"500 CALIFORNIA ST, 2ND FLOOR",San Francisco,CA,94104,37.792888,-122.403135,+14156779494,98.0,20161005,routine,2016
3,24,OMNI S.F. HOTEL - 2ND FLOOR PANTRY,"500 CALIFORNIA ST, 2ND FLOOR",San Francisco,CA,94104,37.792888,-122.403135,+14156779494,96.0,20160311,routine,2016
4,31,NORMAN'S ICE CREAM AND FREEZES,2801 LEAVENWORTH ST,San Francisco,CA,94133,37.807155,-122.419004,NaN,84.0,20160816,routine,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7607,90234,ACCESS SFUSD THE ARC,1520 HOWARD ST,San Francisco,CA,94103,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7608,90236,PHO DONG HUONG,667 MONTEREY BLVD,San Francisco,CA,94127,NaN,NaN,+14154945048,NaN,NaN,NaN,NaN
7609,90237,WIN'S RESTAURANT,3040 TARAVAL ST,San Francisco,CA,94116,NaN,NaN,+14158606036,NaN,NaN,NaN,NaN
7610,90238,USA POWER MARKET,1524 OCEAN AVE,San Francisco,CA,94112,NaN,NaN,+14153331878,NaN,NaN,NaN,NaN


#### Question 6b
Print the 20 lowest rated businesses names, their addresses, and their ratings.

In [125]:
# 20 lowest rated businesses
bus_lowestrated = bus2016_merged[['name', 'address', 'score']]
(bus_lowestrated.sort_values(by='score', ascending=True)).head(20)

,name,address,score
4892,GOLDEN RIVER RESTAURANT,5827 GEARY BLVD,52.0
7325,POKI TIME,2101 LOMBARD ST,55.0
6281,CRAZY PEPPER,2257 SAN JOSE AVE,55.0
4873,UNCLE CAFE,65 WAVERLY PL,55.0
3008,HING WANG BAKERY,339 JUDAH ST,56.0
5627,GOLDEN WOK,295 B ORIZABA AVE,56.0
5383,TOP'S CAFE,2460 SAN BRUNO AVE,57.0
6067,L & G VIETNAMESE SANDWICH,602 EDDY ST,57.0
836,RED A BAKERY,634 CLEMENT ST,57.0
4209,SAN TUNG RESTAURANT LLC,1031 IRVING ST,57.0


## Done!

Now commit this notebook to your `hw3_submissions` branch, push it to your GitHub repo, and open a PR!